# Installation
From [the docs](http://qcodes.github.io/Qcodes/start/index.html):

1. Install [Anaconda](https://www.anaconda.com/download/). Get the Python 3 version. No need to install the VSCode stuff.
1. Save [environment.yml](https://raw.githubusercontent.com/QCoDeS/Qcodes/master/environment.yml) to the machine.
1. Run the following in an Anaconda Prompt:
```
conda env create -f environment.yml
activate qcodes
pip install qcodes
```
1. Close the Anaconda Prompt. You should now be able to open `Jupyter Notebook (qcodes)`.

In [1]:
import qcodes as qc

In [2]:
# This is useful for measurements such as cooling down.
# TODO: Pull this into its own file. Fix the value to be a bit more useful.
import time
class TimeParameter(qc.Parameter):
    def __init__(self, name):
        super().__init__(name, unit='s', label='Epoch Time', docstring='time in seconds since 1970')
        
    def get_raw(self):
        return time.time()
    
    def set_raw(self, val):
        None
tp = TimeParameter('time')

# Instrument setup

At this point, each measurement setup needs to define which instruments they use. See [the docs](http://qcodes.github.io/Qcodes/user/intro.html#instrument).

In [3]:
# For demonstration purposes, use dummy instruments that demonstrate some activated behavior.
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument(name="dac", gates=['ch1', 'ch2', 'ch3', 'ch4'])
dmm = DummyInstrument(name="dmm", gates=['idc', 'ig'])

import math
import random
dmm.idc.get = lambda: 12.2*math.fabs(dac.ch1.get())*(1-math.exp(-(dac.ch1.get()*dac.ch1.get())/3.2))*random.gauss(1, 0.001) + random.gauss(0, 0.001)
dmm.ig.get = lambda: random.gauss(0, 0.01)

In [4]:
station = qc.Station(dac, dmm, tp)

# Measurement

From this point onward, `station.snapshot()` will give the current state of every instrument. You can also call `snapshot()` on individual instruments, as well as parameters. You can use [`qc.Measure()`](http://qcodes.github.io/Qcodes/api/generated/qcodes.measure.Measure.html#qcodes.measure.Measure) to get the current state in a `DataSet`. The results will be saved in the usual place.

In [5]:
_ = qc.Measure(dmm.idc, dmm.ig).run(name='measurement')

A basic sweep looks something like the following:

1. Create a `qc.SweepValues` or `qc.SweepFixedValues` either manually or by calling `sweep(start, stop, step)` on a parameter.
1. Create a `qc.Loop`, set a delay time between steps, and set which values to read each step.
1. Describe what to plot.
1. Run the loop, using a background task to update the plot.

In [6]:
# Sweep dac.ch1 from 0 to 5, 5 to -5, and then -5 to 0.
sv = dac.ch1.sweep(0, 5, 0.1)
sv.extend(dac.ch1.sweep(5, -5, 0.1))
sv.extend(dac.ch1.sweep(-5, 0, 0.1))

# Pause for 0.01 seconds between steps.
loop = qc.Loop(sv, delay=0.01).each(tp, dmm.idc, dmm.ig)
data = loop.get_data_set(name='sweepvg')

# The live plot will show up in a separate window.
# Subplot indices start from 1... yuck!
plot = qc.QtPlot()
plot.add(data.dmm_idc, subplot=1)
plot.add(data.dmm_ig, subplot=2)
plot.add(x=data.time, y=data.dmm_idc, subplot=3)

_ = loop.with_bg_task(plot.update, plot.save).run()

The following "sweep" just watches for a given time. Useful for cooling your sample and so on. Follow [this issue](https://github.com/QCoDeS/Qcodes/issues/131) for more ideas. Hit the interrupt button in jupyter to cancel.

In [7]:
sv = tp.sweep(0, 3600, 0.1)
loop = qc.Loop(sv, delay=0.1).each(tp, dmm.idc, dmm.ig)
data = loop.get_data_set(name='cooldown')
plot = qc.QtPlot()
plot.add(data.dmm_idc, subplot=1)
plot.add(data.dmm_ig, subplot=2)
try:
    _ = loop.with_bg_task(plot.update, plot.save).run()
except KeyboardInterrupt:
    pass

Things to figure out:

* Sequential `qc.Loop`s on one plot. For instance, sweep up gate then sit there for an hour.
* Pause and resume a loop. Verify that interrupts are handled properly.
* How to monitor instrument values while not looping. See [this PR](https://github.com/QCoDeS/Qcodes/pull/534).
* Jupyterlab?

# Data

Data is saved automatically in a file at `DataSet.location`. It is straightforward to configure the location formatter. Plots are also automatically saved. See [the docs](http://qcodes.github.io/Qcodes/dataset/index.html) for more info. There's also an interesting note on [interdependent parameters](http://qcodes.github.io/Qcodes/dataset/interdependentparams.html#).